# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

Description of project: Cournot duopoly

Equations:
Aggregate demand
Individual profit functions (used to find aggregate supply)

Analytical solution:
Find individual response functions

Grid search:
Try different outputs and check that supply equates demand.

Extensions:
Add more periods -> Easier to collude (påkrævet)
Vary delta -> see how collusion is harder to sustain
Vary homogeniety of products
Vary costs - draw from distribution (påkrævet)
Entry/exit 
Simulate model over time - plot quantity/price for different shocks to parameters (Corona-stød?)

Sjove plot (påkrævet)
Set up as class?

In [5]:
import numpy as np
import sympy as sm

from scipy.stats import norm
from scipy import optimize
from numpy import array
import matplotlib.pyplot as plt

%matplotlib inline
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Demand and cost

In [6]:
# Importing necessary libraries
import numpy as np
import scipy.optimize as optimize
import sympy as sm

# Setting up the parameters for the optimization problem
#c = 0.001  # Constant value for the objective function
N = 100  # Number of variables

# Defining symbolic variables for the optimization problem
x = sm.symbols('x')  # Symbolic variable representing x
x_rest = sm.symbols('x_rest')  # Symbolic variable representing x_rest
c=sm.symbols('c')

# Defining the objective function
objective = (1 - x - x_rest - c) * x

# Taking the first derivative of the objective function w.r.t x
obj_dif = sm.diff(objective, x)

# Converting the symbolic expression for the derivative into a callable function
best = sm.lambdify(args=(x, x_rest,c), expr=obj_dif)

# Taking the second derivative of the objective function w.r.t x
best_dif = sm.diff(obj_dif, x)

# Converting the symbolic expression for the second derivative into a callable function
jac_x = sm.lambdify(args=(x, x_rest), expr=best_dif)

# Taking the second derivative of the objective function w.r.t x_rest
best_dif = sm.diff(obj_dif, x_rest)

# Converting the symbolic expression for the second derivative into a callable function
jac_x_rest = sm.lambdify(args=(x, x_rest), expr=best_dif)

# Setting up the initial values for x_rest
x_rest = np.zeros(N)

# Defining the function to be optimized
def h(x):
    y = np.zeros(N)
    for i in range(N):
        # Evaluating the first derivative of the objective function at x[i]
        c_rand = norm.rvs(loc=0, scale=0.001)
        y[i] = best(x[i], sum(x) - x[i],c_rand)
    return y

# Defining the Jacobian of the function to be optimized
def hp(x):
    y = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if j == i:
                # Evaluating the second derivative of the objective function w.r.t x[i]
                y[i,j] = jac_x(x[i], sum(x) - x[i])
            else:
                # Evaluating the second derivative of the objective function w.r.t x_rest[i]
                y[i,j] = jac_x_rest(x[i], sum(x) - x[i])
    return y

# Setting up the initial values for x
x0 = np.array(x_rest)

# Solving the optimization problem using scipy.optimize.root() function
result = optimize.root(h, x0, jac=hp)

# Printing the results
print('\nx =', result.x[0:7], ', h(x) =', h(result.x)[0:7], ',sum(x) =', sum(result.x))


x = [0.01076561 0.0092952  0.01057852 0.00988628 0.0103897  0.0084657
 0.01131106] , h(x) = [-0.00043449  0.00181044 -0.00138068  0.00138153 -0.0004358   0.00153986
 -0.00038314] ,sum(x) = 0.9900372348760774


In [29]:
# Setting up the parameters for the optimization problem
c = 3  # Constant value for the objective function
N = 10  # Number of variables

# Defining symbolic variables for the optimization problem
x = sm.symbols('x')  # Symbolic variable representing x
x_rest = sm.symbols('x_rest')  # Symbolic variable representing x_rest

# Defining the objective function
objective = (1 - x - x_rest - c) * x 

# Taking the first derivative of the objective function w.r.t x
obj_dif = sm.diff(objective, x)

# Converting the symbolic expression for the derivative into a callable function
best = sm.lambdify(args=(x, x_rest), expr=obj_dif)

# Taking the second derivative of the objective function w.r.t x
best_dif = sm.diff(obj_dif, x)

# Converting the symbolic expression for the second derivative into a callable function
jac_x = sm.lambdify(args=(x, x_rest), expr=best_dif)

# Taking the second derivative of the objective function w.r.t x_rest
best_dif = sm.diff(obj_dif, x_rest)

# Converting the symbolic expression for the second derivative into a callable function
jac_x_rest = sm.lambdify(args=(x, x_rest), expr=best_dif)

x_rest = np.zeros(N)

def h(x):
    y = np.zeros(N)
    for i in range(N):  
        y[i] = best(x[i],sum(x)-x[i])
    return y

def hp(x):
    y = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if j == i:
                y[i,j] = jac_x(x[i], sum(x)-x[i])
            else:
                y[i,j] = jac_x_rest(x[i], sum(x)-x[i])
    return y

x0 = np.zeros(N)
bounds = optimize.Bounds(np.zeros(N), np.ones(N))

result = optimize.root(h, x0, jac=hp)

print(result)
print('\nx =', result.x, ', h(x) =', h(result.x))
# Printing the results
print('\nx =', result.x[0:7], ', h(x) =', h(result.x)[0:7], 'sum(x)=', sum(result.x))

 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
       x: [-1.818e-01 -1.818e-01 -1.818e-01 -1.818e-01 -1.818e-01
           -1.818e-01 -1.818e-01 -1.818e-01 -1.818e-01 -1.818e-01]
    nfev: 17
    njev: 2
    fjac: [[-5.219e-01 -3.118e-01 ... -2.499e-01 -2.435e-01]
           [-5.334e-01  8.178e-01 ... -1.950e-02  1.936e-01]
           ...
           [-1.078e-02  2.800e-02 ... -8.904e-01 -1.452e-01]
           [ 4.666e-02 -1.808e-01 ... -1.871e-01  8.686e-01]]
       r: [ 5.241e+00  3.758e+00 ...  1.732e-01 -1.049e+00]
     qtf: [ 6.924e-17 -3.913e-18  2.685e-18  1.500e-16  3.925e-17
            8.606e-17 -6.377e-17 -6.398e-17 -5.467e-17 -4.364e-17]

x = [-0.18181818 -0.18181818 -0.18181818 -0.18181818 -0.18181818 -0.18181818
 -0.18181818 -0.18181818 -0.18181818 -0.18181818] , h(x) = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

x = [-0.18181818 -0.18181

# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 